<a href="https://colab.research.google.com/github/Shivanandroy/T5-Finetuning-PyTorch/blob/main/notebook/T5_Fine_tuning_with_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]

In [ ]:
!conda install -y gdown

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    filelock-3.9.0             |     pyhd8ed1ab_0          13 KB  conda-forge
    g

In [ ]:
!pwd
!pip install nltk

/kaggle/working


In [ ]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)


In [ ]:
trainnews = pd.read_csv("./Book1.csv")

In [ ]:
trainnews.head()

,Headline,Short
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,Ralph Mata was an internal affairs lieutenant ...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,With a breezy sweep of his pen President Vladi...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
#trainnews.drop("Unnamed: 2", axis=1,inplace=True)

In [ ]:
trainnews=trainnews[:100000]

In [ ]:
valnews = pd.read_csv("./validation.csv")

In [ ]:
trainnews.columns=['article','highlights']

In [ ]:
trainnews.sample(10)

,article,highlights
56032,Elizabeth Edwards tormented herself by using v...,"Tell-all memoir, 'What Really Happened: John E..."
35773,An elderly woman driver escaped serious injury...,"The female driver, who is in her 80s, escaped ..."
6732,The highest paid female business executive in ...,"Martine Rothblatt, 60, was married father-of-f..."
34425,By . Mail Foreign Service . PUBLISHED: . 16:02...,Palestinians: Move casts doubt on Israel's 'si...
90282,England new boy Nathaniel Clyne looked at home...,Wayne Rooney leads England training ahead of E...
62146,This week's bitter budget showdown in Wisconsi...,Governor can't get a quorum in Wisconsin Senat...
68149,"Bronze Age Brits were wearing bling around 4,0...","The 4,000-year-old necklace was found in an ar..."
8584,"By . Matt Chorley, Mailonline Political Editor...",Catalogue of equipment stolen from April to Se...
75505,Rangers caretaker boss Kenny McDowall has sens...,Rangers have loaned five players from Newcastl...
73829,Hundreds of dolphins are seen diving through t...,"The dolphins migrate from Mossel Bay, South Af..."


In [ ]:
valnews.sample(10)

,id,article,highlights
8238,a02c833b59349282e09212a2e6c6ecd308ea3e46,"No-one enjoys filing their tax return, but pay...",Tax receipt written in Greek on pottery is dat...
9274,c02bf0991fa612825581857f246c29a7b5820083,(CNN)A Georgia police chief who said he accide...,William McCollom says he accidentally shot wif...
12860,44e182d721c6d78267b3e288e886b4efd6abc693,Cancel your plans to journey through the centr...,Originally it was thought the journey from one...
9971,dfe8ae0b69179fb9ebe7a81a24ba0baded2e8323,Three police officers have been questioned as ...,"EXCLUSIVE: Poppi Worthington, from Barrow-in-F..."
889,762292391425a10d9dec2fdd077fe2aaf02021fa,A north Texas woman is outraged with her five-...,"Gabriel Muntu, 5, was supposed to remain at sc..."
5486,6744e427592c9a29a68051c861b0b26139ade3d2,Newcastle have announced record profits of £18...,Newcastle have announced record profits of £18...
771,fa1a65db1127880f704ff189754e1123d164ba7f,(CNN)Once again the global community waits to ...,Authors: Iran's people want a deal in hopes of...
3029,fbd22d5f685ec8d006201c41e36fd092e26d6bd3,(CNN)There's a steady stream of blood flowing ...,"The documentary ""Dawg Fight"" shows men fightin..."
10363,581c1a4bd166e8571fec9799d2e47b72024c54f5,If you want the memory of your loved ones or f...,The Bios Urn has been developed by Barcelona-b...
6552,b9550576c750440c1cbf2a6a3e6d706e71739eea,This is the moment an abandoned dog is rescued...,Video footage shows animal rescue worker Eldad...


In [ ]:
valnews.drop("id",axis=1,inplace=True)

In [ ]:
valnews.sample(10)

,article,highlights
4553,Washington (CNN)House Speaker John Boehner sai...,House Speaker says his problem with Bergdahl c...
5315,"England head coach Peter Moores, who should su...",Charles Sale: Peter Moores should be sacked af...
12702,An airline has paid an undisclosed amount to m...,Financial details of the settlements remain co...
7298,"(CNN)The ""Fifty Shades"" series will have to ha...","Sam Taylor-Johnson bows out of the ""Fifty Shad..."
7200,"April 1, 2015 . A trip around the world starts...",This page includes the show Transcript .\nUse ...
8897,A couple are facing jail after they blackmaile...,Chauffeur Awad Abdulbagy drove family's photog...
6979,It was once a symbol of U.S. supremacy before ...,Former U.S. embassy in Tehran is a chilling mu...
8270,A convicted robber has sparked outrage after p...,Adam Ali is currently serving a seven year sen...
1314,Ryanair has backtracked on plans to develop a ...,Ryanair had announced plans for a service betw...
11585,Imagine playing a game of Pictionary with a gr...,"Suki Kim, a Korean-American journalist, taught..."


In [ ]:
trainnews["article"] = "summarize: "+trainnews["article"]

In [ ]:
trainnews.head()

,article,highlights
0,summarize: By . Associated Press . PUBLISHED: ...,"Bishop John Folda, of North Dakota, is taking ..."
1,summarize: Ralph Mata was an internal affairs ...,Criminal complaint: Cop used his role to help ...
2,summarize: A drunk driver who killed a young w...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,summarize: With a breezy sweep of his pen Pres...,Nina dos Santos says Europe must be ready to a...
4,summarize: Fleetwood are the only team still t...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
valnews["article"] = "summarize: "+valnews["article"]

In [ ]:
valnews.head()

,article,highlights
0,"summarize: Sally Forrest, an actress-dancer wh...","Sally Forrest, an actress-dancer who graced th..."
1,summarize: A middle-school teacher in China ha...,Works include pictures of Presidential Palace ...
2,summarize: A man convicted of killing the fath...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,summarize: Avid rugby fan Prince Harry could b...,Prince Harry in attendance for England's crunc...
4,summarize: A Triple M Radio producer has been ...,Nick Slater's colleagues uploaded a picture to...


In [ ]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
device

'cuda'

In [ ]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%1000==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=3.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%1000==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [ ]:
#def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/" ):
def T5Trainer(trainnews, valnews, source_text, target_text, model_params, output_dir="./outputs/" ):
  
  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)
  
  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  #dataframe = dataframe[[source_text,target_text]]
  trainnews= trainnews[[source_text,target_text]]
  valnews=valnews[[source_text,target_text]]
  #display_df(dataframe.head(2))
  display_df(trainnews.head(2))
  display_df(valnews.head(2))

  
  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  #train_size = 0.8
  #train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  #val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  #train_dataset = train_dataset.reset_index(drop=True)
  train_dataset = trainnews.reset_index(drop=True)
  val_dataset = valnews.reset_index(drop=True)


  #console.print(f"FULL Dataset: {dataframe.shape}")
  #console.print(f"TRAIN Dataset: {train_dataset.shape}")
  #console.print(f"TEST Dataset: {val_dataset.shape}\n")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)



  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)
        #pass
      
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  #path = os.path.join(output_dir, "model_files")
  #model.save_pretrained(path)
  model.save_pretrained(output_dir)
  tokenizer.save_pretrained(output_dir)
  #model_dir="./model_files/"
  #model_dir="."  
  #tokenizer = T5Tokenizer.from_pretrained(model_dir)
  #model = T5ForConditionalGeneration.from_pretrained(model_dir)
  model.cuda()


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
  console.save_text(os.path.join(output_dir,'logs.txt'))
  
  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [ ]:
model_params={
    "MODEL" : "t5-small",
    "TRAIN_BATCH_SIZE":16,          # training batch size
    "VALID_BATCH_SIZE":16,          # validation batch size
    "TRAIN_EPOCHS":5,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":64,   # max length of target text
    "SEED": 42                     # set seed for reproducibility 

}

In [ ]:
T5Trainer(trainnews, valnews, source_text="article", target_text="highlights", model_params=model_params, output_dir=".")

[20:01:14] [Model]: Loading t5-small...                                                            ]8;id=927228;file:///tmp/ipykernel_23/3469013183.py\3469013183.py]8;;\:]8;id=897924;file:///tmp/ipykernel_23/3469013183.py#15\15]8;;\
                                                                                                                   

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

...

Completed 0

[02:18:22] [Validation Completed.]                                                                ]8;id=136275;file:///tmp/ipykernel_23/3469013183.py\3469013183.py]8;;\:]8;id=247740;file:///tmp/ipykernel_23/3469013183.py#114\114]8;;\
                                                                                                                   

[Model] Model saved @ ./model_files

[Validation] Generation on Validation data saved @ ./predictions.csv

[Logs] Logs saved @ ./logs.txt

In [ ]:
model_dir="."

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)
model.cuda()

#T5Tokenizer, T5ForConditionalGeneration

In [ ]:
source_text="article"
target_text='highlights'
trainnews= trainnews[[source_text,target_text]]
valnews=valnews[[source_text,target_text]]
  #display_df(dataframe.head(2))
display_df(trainnews.head(2))
display_df(valnews.head(2))

  
# Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  #train_size = 0.8
  #train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  #val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  #train_dataset = train_dataset.reset_index(drop=True)
train_dataset = trainnews.reset_index(drop=True)
val_dataset = valnews.reset_index(drop=True)


  #console.print(f"FULL Dataset: {dataframe.shape}")
  #console.print(f"TRAIN Dataset: {train_dataset.shape}")
  #console.print(f"TEST Dataset: {val_dataset.shape}\n")
console.print(f"TRAIN Dataset: {train_dataset.shape}")
console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  


  # Defining the parameters for creation of dataloaders
train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)
output_dir='./outputs/'
for epoch in range(1):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
console.save_text(os.path.join(output_dir,'logs.txt'))
  
console.log(f"[Validation Completed.]\n")
console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [ ]:
final_df

In [ ]:
final_df.to_csv(os.path.join('./outputs/','predictions.csv'))
  
console.save_text(os.path.join('./outputs/','logs.txt'))